# Data cleaning

## Setup

In [1]:
# import modules
import pandas as pd
import numpy as np
import rootpath
import re
import qgrid
import glob

# Set root path for project
path = rootpath.detect()

# Read in data
# Use list comprehension to read in all files

# Oily skin
df_cleanser_oily = pd.concat([pd.read_csv(file, index_col=0) for file in glob.glob(f"{path}/data/cleansers_face-wash_oilyskin_pg*.csv")], ignore_index=True)
# Dry skin
df_cleanser_dry = pd.concat([pd.read_csv(file, index_col=0) for file in glob.glob(f"{path}/data/cleansers_face-wash_dryskin_pg*.csv")], ignore_index=True)
# Normal skin
df_cleanser_normal = pd.concat([pd.read_csv(file, index_col=0) for file in glob.glob(f"{path}/data/cleansers_face-wash_normalskin_pg*.csv")], ignore_index=True)


## Examine data

In [25]:
df_cleanser_oily.shape

(238, 14)

In [26]:
df_cleanser_dry.shape

(208, 14)

In [27]:
df_cleanser_normal.shape

(267, 14)

In [211]:
df_cleansers = pd.concat([df_cleanser_oily,df_cleanser_dry,df_cleanser_normal], axis=0)
#df_cleansers.shape

In [29]:
qgrid.show_grid(df_cleansers, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [11]:
df_cleansers.describe()

,ratings,page
count,95.000000,95.0
mean,4.470526,1.0
std,0.234690,0.0
min,3.900000,1.0
25%,4.350000,1.0
50%,4.500000,1.0
75%,4.700000,1.0
max,5.000000,1.0


In [15]:
df_cleansers.dtypes

use_category        object
use_subcategory     object
skintype            object
brand               object
product             object
size                object
price               object
details             object
ingredients         object
ratings            float64
perc_respondrec     object
total_reviews       object
link                object
page                 int64
dtype: object

## Data cleaning

### Remove rows with problematic data

In [212]:
# Filter rows without size information
# df_cleansers[df_cleansers['size'].str.contains('Item')]

# Remove rows with no size information
df_cleansers.drop(df_cleansers[df_cleansers['size'].str.contains('Item')].index, inplace=True)
df_cleansers.drop(df_cleansers[df_cleansers['size'].str.contains('ounce')].index, inplace=True)
df_cleansers.drop(df_cleansers[df_cleansers['size'].str.contains('Original')].index, inplace=True)
df_cleansers.drop(df_cleansers[df_cleansers['size'].str.contains(' ct')].index, inplace=True)

# Remove rows with no ingredient information
df_cleansers.dropna(subset=['ingredients'], inplace = True)

# Remove specific row
df_cleansers.drop(df_cleansers[df_cleansers['ingredients'].str.contains('This item is not eligible for coupon discounts')].index, inplace=True)

# Remove travel size items
df_cleansers.drop(df_cleansers[df_cleansers['product'].str.contains('Travel Size')].index, inplace=True)

In [4]:
qgrid.show_grid(df_cleansers, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

### Regex

In [159]:
pd.set_option('display.max_seq_items', 2000)
pd.set_option('display.max_rows', 2000)

In [317]:
df2 = df_cleansers.copy()
# FOR ALL: CHECK dtype AFTER MANIPULATION

# Size
# Remove oz and select original price only

# Check unique sizes
#df_cleansers['size'].unique()

# Remove 'size' and 'oz|'
df2['size'] = df2['size'].str.replace('Size | oz| oz\||\|','').astype('float64')

# Remove price text
#df_cleansers['price'].unique()
df2['price'] = df2['price'].str.replace('Price\\n\$|Sale Price\\n\$\d*\.\d*\\nOriginal Price\\n\$|Price\\n\$|\\nMarket Value\\n\(\$39 value\)','').astype('float64')

# Remove reviews text
df2['total_reviews'] = df2['total_reviews'].str.replace(' Reviews| Review','')#.astype('Int64')#.astype(float).astype(int)
df2['total_reviews'] = pd.to_numeric(df2['total_reviews'],errors='coerce',downcast='signed')
df2['total_reviews'] = df2['total_reviews'].astype('Int64')


# Get the first ingredient after splitting into strings to figure out water regex
# Change string of ingredients into a list containing a string for each ingredient, split by the following characters
df2.ingredients = df2.ingredients.str.split(r',|\\n') # |\.
## NEED TO FIX PERIOD B/C AFFECTS ACTIVE INGREDIENTS

# strip whitespace from around each ingredient
df2.ingredients = [[ingredient.strip().lower() for ingredient in product] for product in df2.ingredients]
df2 = df2.explode('ingredients')

# for ingredient in df2.ingredients:
#     if 'water' in ingredient:
#         df2.newcol = 'yes'
#     else:
#         df2.newcol = 'no'

# Replace all rows that contain string water with water, get rid of alternative spellings
df2['ingredients'] = np.where(df2['ingredients'].str.contains('water'), 'water', df2['ingredients'])  
# Same for fragrance
df2['ingredients'] = np.where(df2['ingredients'].str.contains('fragrance'), 'fragrance', df2['ingredients'])  
        
test = df2['ingredients'].unique()

# Create column for presence of ingredient in a product (excluding absences and does not account for all possible ingredients)
df2['ingred_value'] = 1
df2['skintype_value'] = 1  
## CHANGE INTO REVERSE SEQUENCE FOR ORDINAL ENCODING LATER

#df2.ingredients[:10]
# Replace strings that contain 'water' with water
# for product in df2.ingredients:
#     for ingredientlist in product:
#         for ingredient in ingredientlist:
#             if 'water' in ingredient:
#                 ingredient = ingredient.replace('^.*(water).*','water')
#                 product[ingredientlist][ingredient] = ingredient.replace(ingredient,'water')
#df2.ingredients[:10]
# for product in df2.ingredients:
#     for ingredientlist in product:
#             if 'water' in ingredientlist:
#                 product[ingredientlist] = ingredientlist.replace(ingredientlist,'water')
#         for ingredient in ingredientlist:
#             if 'water' in ingredient:
#                 df2.ingredients[i] = 'water'
#df2.ingredients.iloc[1]#[0]
#df2.ingredients = [product.split(',|\\n|\.') for product in df2.ingredients]
#df_cleansers['ingredients'] = [[ingredient for ingredient in str(product).split(', |\\n|\.')] for product in df_cleansers.ingredients]
#df2.ingredients=[[ingredient.split(', ') for ingredient in product] for product in df2.ingredients]
# Separate ingredient lists into lists of strings for each product
#test = [[ingredient.lower() for ingredient in product] for product in test] # df_cleansers.ingredients
#df_cleansers.ingredients = [[ingredient.lower() for ingredient in product] for product in df_cleansers.ingredients]
#df2.ingredients.str.split(',')
#test[:5]
# Look at first ingredient of each product for reformatting water
#test = [ingredient[0] for ingredient in df_cleansers.ingredients]
#np.unique(test)

# test2 = [ingredient.str.replace('\(([^)]+)\)| \/ aqua \/ eau|\/aqua\/eau|\\\\aqua\\\\eau|' \
#                                '\(mineral spring water\)| purified| \(solvent\/diluent\)|' \
#                                ' \(water/eau\)|') for ingredient in df_cleansers.ingredients]
# test2 = [[ingredient.replace('.*(water).*', '') for ingredient in product] for product in df2.ingredients]
# test2
#df2.ingredients[:10]
# test1 = df2.ingredients.iloc[1]
# test3 = test1[0].replace('.*(water).*', '')
# test3
# for i, item in enumerate(list_of_strings):
#     if "super" in item:
#         list_of_strings[i] = "superintendent"
#np.unique(test2)
#df_cleansers.ingredients.iloc[0]
#df_cleansers['ingredients']
#df_cleansers['ingredients'].unique()


# Percent responded would recommend
# Remove percent symbol
# df_cleanser_oily['perc_respondrec'] = df_cleanser_oily['perc_respondrec'].str.replace('\%', '').astype(int)
# df_cleanser_oily['perc_respondrec'].head()

# Total Reviews #
# Remove whitespace and 'reviews'
# df_cleanser_oily['total_reviews'] = df_cleanser_oily['total_reviews'].str.replace(' Reviews', '').astype(int)
# df_cleanser_oily['total_reviews'].head()
df2.head()

,use_category,use_subcategory,skintype,brand,product,size,price,details,ingredients,ratings,perc_respondrec,total_reviews,link,page,ingred_value,skintype_value
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,water,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,sodium laureth sulfate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,cocamidopropyl betaine,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,jojoba esters,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,acrylates copolymer,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1


In [327]:
# Add sequence for each ingredient in product
df2['ingred_order'] = df2.groupby('product')['ingred_value'].rank(method="first", ascending=True)

# Get max value of sequence and store in separate df
df2temp = df2.groupby('product')['ingred_order'].max().reset_index()
# Plus 1 so that last ingredient is 1 when take difference of max and ingredient order
# Otherwise zero will indicate that last ingredient isn't present
df2temp['ingred_order'] = df2temp['ingred_order']+1
df2temp=df2temp.rename(columns={"ingred_order":"ingred_ordinal_max"})

df2temp.head(n=10)        
#df2.head(n=10)

,product,ingred_ordinal_max
0,1000 Roses Cleansing Foam,30.0
1,3% Glycolic Solutions Cleanser,49.0
2,82% Organic Softening Cleanser,43.0
3,AHA/BHA Acne Clearing Gel,34.0
4,AHA/BHA Exfoliating Cleanser,43.0
5,AM To The PM Gentle Gel Cleanser,136.0
6,Acne Clearing Wash,91.0
7,Acne Deep Pore Cleansing Wash,33.0
8,Acne Relief Gentle & Soothing Cleanser,25.0
9,Acne Solutions Cleansing Bar for Face & Body,58.0


In [328]:
# Merge with original DF
df2=pd.merge(df2,df2temp,on='product')
df2.head()

,use_category,use_subcategory,skintype,brand,product,size,price,details,ingredients,ratings,perc_respondrec,total_reviews,link,page,ingred_value,skintype_value,ingred_order,ingred_ordinal_max
0,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,water,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,1.0,43.0
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,sodium laureth sulfate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,2.0,43.0
2,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,cocamidopropyl betaine,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,3.0,43.0
3,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,jojoba esters,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,4.0,43.0
4,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.0,Featuring limited-edition Hydration packaging ...,acrylates copolymer,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,5.0,43.0


In [329]:
# Take difference of ordinal max from ingredient order to get ordinal values for ingredients
df2['ingred_ordinal'] = df2["ingred_ordinal_max"] - df2["ingred_order"]
df2.head(n=45)

,use_category,use_subcategory,skintype,brand,product,size,price,details,ingredients,ratings,perc_respondrec,total_reviews,link,page,ingred_value,skintype_value,ingred_order,ingred_ordinal_max,ingred_ordinal
0,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,water,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,1.0,43.0,42.0
1,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,sodium laureth sulfate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,2.0,43.0,41.0
2,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,cocamidopropyl betaine,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,3.0,43.0,40.0
3,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,jojoba esters,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,4.0,43.0,39.0
4,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,acrylates copolymer,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,5.0,43.0,38.0
5,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,glycol stearate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,6.0,43.0,37.0
6,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,butylene glycol,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,7.0,43.0,36.0
7,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,sodium pca,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,8.0,43.0,35.0
8,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,dipotassium glycyrrhizate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,9.0,43.0,34.0
9,cleanser,face wash,oily,Murad,AHA/BHA Exfoliating Cleanser,6.75,40.00,Featuring limited-edition Hydration packaging ...,sodium ascorbyl phosphate,4.6,93%,267,https://www.ulta.com/ahabha-exfoliating-cleans...,1,1,1,10.0,43.0,33.0


In [221]:
df2.head()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

#### Pivot long to wide

In [370]:
df3 = df2.pivot_table(index=['product','skintype','skintype_value'],
                    columns='ingredients',
                    values='ingred_ordinal',
                     aggfunc='max',
                     fill_value=0)

In [371]:
# Put index values back as columns
df3.reset_index(inplace=True)

In [372]:
df3.head()

ingredients,product,skintype,skintype_value,,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,...,zinc gluconate,zinc laurate,zinc oxide (mineral),zinc pca,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root
0,1000 Roses Cleansing Foam,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3% Glycolic Solutions Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3% Glycolic Solutions Cleanser,normal,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3% Glycolic Solutions Cleanser,oily,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,82% Organic Softening Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
# Shows all unique ingredients, use to find errors in input
print (df3.columns.tolist())

['', '(thyme) leaf extract', '* aloe barbadensis (aloe vera) leaf juice', '* cymbopogon schoenanthus (lemongrass) oil', '* potassium cocotte (saponified coconut oil*)', '* rosmarinus officials (rosemary) leaf oil', '* tocopherol (vitamin e). *certified usda organic', '*95% naturally derived/dérivé naturellement.', '*avena sativa (oat) kernel extract', '*baptisia tinctoria root extract', '*calendula officinalis flower extract', '*citronellol', '*geraniol', '*limonene', '*linalool. *natural constituent of essential oils listed.', '*nepeta cataria extract', '*rubus idaeus (raspberry) leaf extract', '*stellaria media (chickweed) extract', '1', '10-hydroxydecanoic acid', '100% natural ingredients: sodium cocoate**', '100% natural ingredients:\xa0sodium cocoate**', '2 hexanediol', '2-hexanediol', '^limonene', '^linalool.\xa0 ^natural constituent of essential oils listed.', 'abies sibirica oil', 'acacia concinna fruit extract', 'acacia decurrens (mimosa) flower wax', 'acacia senegal gum', 'ac

In [373]:
# Pivot wider based on skintype
df4 = df3.pivot_table(index='product',
                    columns='skintype',
                    values='skintype_value',
                     aggfunc='max',
                     fill_value=0)

In [368]:
df4.head()

skintype,dry,normal,oily
product,,,
1000 Roses Cleansing Foam,1,0,0
3% Glycolic Solutions Cleanser,1,1,1
82% Organic Softening Cleanser,1,1,0
AHA/BHA Acne Clearing Gel,1,1,1
AHA/BHA Exfoliating Cleanser,0,1,1


In [374]:
# Put index values back as columns
df4.reset_index(inplace=True)

In [248]:
print (df4.columns)

Index(['product', 'dry', 'normal', 'oily'], dtype='object', name='skintype')


In [375]:
# merge df for one hot encoding for skintypes
df5=pd.merge(df3,df4,on='product')

In [376]:
df5.head()

,product,skintype,skintype_value,,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,...,zinc pca,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily
0,1000 Roses Cleansing Foam,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3% Glycolic Solutions Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
2,3% Glycolic Solutions Cleanser,normal,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,3% Glycolic Solutions Cleanser,oily,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,82% Organic Softening Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [339]:
# Get rid of skintype and skintype_value columns now that they're one hot encoded
df5.drop(columns=['skintype','skintype_value'], inplace=True)
df5.head()

,product,,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,*95% naturally derived/dérivé naturellement.,*avena sativa (oat) kernel extract,...,zinc pca,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily
0,1000 Roses Cleansing Foam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3% Glycolic Solutions Cleanser,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
2,3% Glycolic Solutions Cleanser,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,3% Glycolic Solutions Cleanser,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,82% Organic Softening Cleanser,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [377]:
# Extract distinct rows since no longer need product duplicates
df5=df5.drop_duplicates(subset = ["product"])
df5.head()

,product,skintype,skintype_value,,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,...,zinc pca,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily
0,1000 Roses Cleansing Foam,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3% Glycolic Solutions Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,82% Organic Softening Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
6,AHA/BHA Acne Clearing Gel,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
9,AHA/BHA Exfoliating Cleanser,normal,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [378]:
# Add total ingredients column
tempdf = df2[['ingred_ordinal_max', 'product']]
tempdf = tempdf.rename(columns={'ingred_ordinal_max':'total_ingred'})
# Remove extra 1 added for ordinal encoding
tempdf['total_ingred']=tempdf['total_ingred']-1
# Remove duplicate rows
tempdf=tempdf.drop_duplicates(subset = ["product"])
#tempdf.head()
# Merge df
df6 = df5.merge(tempdf, how = 'left', on = 'product')
df6.head()

,product,skintype,skintype_value,,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,...,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily,total_ingred
0,1000 Roses Cleansing Foam,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,29.0
1,3% Glycolic Solutions Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,48.0
2,82% Organic Softening Cleanser,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,42.0
3,AHA/BHA Acne Clearing Gel,dry,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,33.0
4,AHA/BHA Exfoliating Cleanser,normal,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,42.0


In [382]:
# Export data for analysis
df6.to_csv(f"{path}/data/data_clean.csv")
df6.shape

(232, 1922)

In [384]:
X = np.array([[2, 3], [3, 5], [5, 8]])
Y = np.array([[1, 0], [2, 1]])